In [ ]:
!pip install transformers
!pip install keras
!pip install Sentencepiece
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install datasets
!pip install -U scikit-learn

  Using cached sentencepiece-0.1.97-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["WANDB_DISABLED"] = "true"

import transformers
import torch
#import keras
from datasets import load_dataset, concatenate_datasets

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from functions import *

from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer

In [ ]:
ds_list = ["instagram", "nonsmoking", "synthetic", "virtualcoach", "WoZ", "test_drop", "optifast"]
classif_list = ["function_Sentence", "nonfunction_Sentence", "non_noun_adj"]

In [ ]:
preds = []
testset = []
classifier = []
ds_size = []

for index, ds in enumerate(ds_list):
    to_load = "forum/"+ds+"_label_preproc.csv"
    forum_test = load_predict_testset(to_load, "Sentence", model_path="../MI_Data/Bert_Finetuning/models/labels/model")
    result = compute_test_metrics(forum_test, 'macro')
    preds.append(result["f1"])
    testset.append(ds)
    classifier.append("original")
    forum_test = load_predict_testset(to_load, "Sentence", model_path="../MI_Data/Bert_Finetuning/models/labels_undersampled/model")
    result = compute_test_metrics(forum_test, 'macro')
    preds.append(result["f1"])
    testset.append(ds)
    classifier.append("original_undersampled")
    ds_size.append(len(ds))
    for jindex, classif in enumerate(classif_list):
        path = "classif/"+classif+"/model"
        forum_test = load_predict_testset(to_load, classif, model_path=path)
        result = compute_test_metrics(forum_test, 'macro')
        preds.append(result["f1"])
        testset.append(ds)
        classifier.append(classif)
    

In [ ]:
d = {"Testset":testset, "classifier":classifier, "f1 score":preds}
resultslist = pd.DataFrame(d)

In [ ]:
resultslist.tail()

In [ ]:
resultslist.to_csv("classification_results_ablations.csv")